In [ ]:
import os
import json
import sys

## Logging

In [ ]:
# Set up logging
import logging
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [ ]:
import tiktoken
from llama_index.callbacks import CallbackManager, TokenCountingHandler
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext

# you can set a tokenizer directly, or optionally let it default
# to the same tokenizer that was used previously for token counting
# NOTE: The tokenizer should be a function that takes in text and returns a list of tokens
token_counter = TokenCountingHandler(
    tokenizer=tiktoken.encoding_for_model("text-davinci-003").encode,
    verbose=False  # set to true to see usage printed to the console
    )

## Read and set API keys

In [ ]:
# Open and read the config file
with open('config.json', 'r') as config_file:
    config_data = json.load(config_file)

# Retrieve the API key from the config data
api_key = config_data['api_key']
os.environ['OPENAI_API_KEY'] = api_key

## Load the index

In [ ]:
from llama_index import StorageContext, load_index_from_storage

# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="./storage")
# load index
index = load_index_from_storage(storage_context)

In [ ]:
chat_engine = index.as_chat_engine()
response = chat_engine.chat("what is LiquidEarth?")
print(response)

## GPT 4

In [ ]:
from llama_index.llms import OpenAI
from llama_index import ServiceContext

gpt4 = OpenAI(temperature=0, model="gpt-4")
service_context_gpt4 = ServiceContext.from_defaults(llm=gpt4)

### create Prompt Template

In [ ]:
from llama_index import Prompt
# define custom Prompt
TEMPLATE_STR = (
    "You are Leia, the LiquidEarth Intelligent Assistant. You are helping a user with a question about LiquidEarth. You are very smart and friendly and always in a great mood.\n"
    "In LiquidEarth, a 'Space' and a 'Project are the same thing. We have provided Documentation on the software and further information below. In some cases the metadata includes a 'Control' Field that points to a UI element in the app associated to the described functionality. this is only for internal use. when describing controls to the user, use descriptions and names from the text, not the 'control' values. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Answer the question for a human to understand. Additionally, return the 'Control' properties in the order of operations in the following form at the end of your response: [[control1], [control2], ...]. Append the list to your response without further comment. If no controls are found, do not comment it. Never include any controls that are not specified in the Metadata Field in the provided documentation. Do not interpret any controls from the text body. If the answer requires multiple steps, describe each step in detail. Given this information, please answer the question: {query_str}\n"
)
QA_TEMPLATE = Prompt(TEMPLATE_STR)

In [ ]:
query_engine = index.as_query_engine(service_context=service_context_gpt4, text_qa_template=QA_TEMPLATE, retriever_mode="embedding")

In [ ]:
#dirty hack: trying to increase the context size
query_engine.retriever._similarity_top_k = 6

In [ ]:
response = query_engine.query("hello,how can i create a project and add some data?")

In [ ]:
print(response)

In [ ]:
response


In [ ]:
response

## Chat with a prompt template (ToDo)

In [ ]:
custom_prompt = Prompt("""\
Given a conversation (between Human and Assistant) and a follow up message from Human, \
rewrite the message to be a standalone question that captures all relevant context \
from the conversation.

<Chat History>
{chat_history}

<Follow Up Message>
{question}

<Standalone question>
""")

# list of (human_message, ai_message) tuples
custom_chat_history = [
    (
        'Hello assistant, we are having a insightful discussion about Paul Graham today.',
        'Okay, sounds good.'
    )
]

query_engine = index.as_query_engine()
chat_engine = CondenseQuestionChatEngine.from_defaults(
    query_engine=query_engine,
    condense_question_prompt=custom_prompt,
    chat_history=custom_chat_history,
    verbose=True
)

## print token usage

In [ ]:
print('Embedding Tokens: ', token_counter.total_embedding_token_count, '\n',
      'LLM Prompt Tokens: ', token_counter.prompt_llm_token_count, '\n',
      'LLM Completion Tokens: ', token_counter.completion_llm_token_count, '\n',
      'Total LLM Token Count: ', token_counter.total_llm_token_count)